In [1]:
%cd SELFRec/

/workspace/viettel-ai-challenge-track-ds/SELFRec


/opt/conda/envs/rapids-24.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from util.conf import ModelConf
from data.loader import FileIO
import torch
import torch.nn as nn
import torch.nn.functional as F
from base.graph_recommender import GraphRecommender
# from util.conf import OptionConf
from util.sampler import next_batch_pairwise
from base.torch_interface import TorchGraphInterface
from util.loss_torch import bpr_loss, l2_reg_loss, InfoNCE

In [160]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/training_set.csv')
df = df[["UserId", "ItemId", "Purchase"]]

user_list = df.groupby('UserId')['ItemId'].nunique()
user_list_denoise = user_list[(user_list>=2) & (user_list<=500000)].index.to_list()

In [161]:
test = pd.read_csv('../data/public_testset.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
test_user_id = test['user_id'].values

In [162]:
final_user_list = []
final_user_list.extend(user_list_denoise)
final_user_list.extend(test_user_id)
user_list_denoise = list(set(final_user_list))

In [163]:
df = df[df.UserId.isin(user_list_denoise)]

In [164]:
inference_df = df

In [165]:
from tqdm.notebook import tqdm
session_items = []
users = []
current_user = df.iloc[0]["UserId"]
current_items = []
for _, i in tqdm(df.iterrows(), total=len(df)):
    if i["UserId"] != current_user:
        session_items.append(current_items)
        users.append(current_user)
        current_user = i["UserId"]
        current_items = []

    current_items.append(i["ItemId"])
    if i["Purchase"] == 1:
        session_items.append(current_items)
        users.append(current_user)
        current_items = []

  0%|          | 0/389923 [00:00<?, ?it/s]

In [166]:
session = pd.DataFrame(zip(session_items, users), columns=["ItemId", "TrueUser"]).reset_index().rename(columns={"index": "UserId"}).explode("ItemId")
session

,UserId,ItemId,TrueUser
0,0,tyviMi4b8Q,tyviMi4b8Q
0,0,t3fB9Nq1VY,tyviMi4b8Q
0,0,X7ZdDwPBKM,tyviMi4b8Q
0,0,r4bieaZn66,tyviMi4b8Q
0,0,cRl2I3cVGB,tyviMi4b8Q
...,...,...,...
55199,55199,05N12YIBr4,Uo2X1ah0bf
55199,55199,hRapAMLApl,Uo2X1ah0bf
55199,55199,6S3euGhLbW,Uo2X1ah0bf
55199,55199,56em7Cl1uk,Uo2X1ah0bf


In [167]:
# df = df[df.UserId.isin(user_list_denoise)]
df = session
# Split data into train and test sets (90% for train, 10% for test)
train_df = df.sample(frac=0.95, random_state=42)  # 90% for train
# train_df = df
test_df = df.drop(train_df.index)  # Remaining 10% for test

# Save train and test sets as .txt files without headers, separated by a space
train_df.to_csv("train.txt", index=False, header=False, sep=" ")
test_df.to_csv("test.txt", index=False, header=False, sep=" ")

In [168]:
# user_map = {UserId: index for index, UserId in enumerate(user_list_denoise)}
# user_map = pd.DataFrame(list(user_map.items()), columns=['UserId', 'index'])
# user_map.head()

In [169]:
df["UserId"].unique()

array([    0,     1,     2, ..., 55197, 55198, 55199])

In [170]:
user_map = {UserId: index for index, UserId in enumerate(df["UserId"].unique())}
user_map = pd.DataFrame(list(user_map.items()), columns=['UserId', 'index'])
user_map.head()

,UserId,index
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [171]:
item_list = df.ItemId.unique()
item_map = {ItemId:index for index, ItemId in enumerate(item_list)}
item_map = pd.DataFrame(list(item_map.items()), columns=['ItemId', 'index'])
item_map.head()

,ItemId,index
0,tyviMi4b8Q,0
1,t3fB9Nq1VY,1
2,X7ZdDwPBKM,2
3,r4bieaZn66,3
4,cRl2I3cVGB,4


In [172]:
df['UserId'] = df.UserId.map(dict(user_map[['UserId', 'index']].values))
df['ItemId'] = df.ItemId.map(dict(item_map[['ItemId', 'index']].values))
df.head()

,UserId,ItemId,TrueUser
0,0,0,tyviMi4b8Q
0,0,1,tyviMi4b8Q
0,0,2,tyviMi4b8Q
0,0,3,tyviMi4b8Q
0,0,4,tyviMi4b8Q


In [173]:
df_by_user = df.groupby('UserId')['ItemId'].apply(list).reset_index()
df_by_user.head()

,UserId,ItemId
0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,"[24, 25, 26, 27, 28]"
2,2,"[29, 30, 31, 32, 33, 34, 35]"
3,3,"[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 4..."
4,4,[50]


In [174]:
user_list_test = user_map.copy()
user_list_test
train_df = []
test_df = []
for _, row in df_by_user.iterrows():
    user_id = row['UserId']
    item_ids = row['ItemId']
    
    if user_id in user_list_test['index'].values:
        # Split last item into test_df and the rest into train_df
        train_df.append({'UserId': user_id, 'ItemId': item_ids[:-1]})
        test_df.append({'UserId': user_id, 'ItemId': [item_ids[-1]]})
    else:
        # All items go to train_df if user is not in user_list_test
        train_df.append({'UserId': user_id, 'ItemId': item_ids})

In [175]:
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)

In [176]:
with open("train_sequential.txt", "w") as file:
    for _, row in train_df.iterrows():
        user_index = row['UserId']
        item_indices = ' '.join(map(str, row['ItemId']))
        file.write(f"{user_index}:{item_indices}\n")

In [177]:
with open("test_sequential.txt", "w") as file:
    for _, row in test_df.iterrows():
        user_index = row['UserId']
        item_indices = ' '.join(map(str, row['ItemId']))
        file.write(f"{user_index}:{item_indices}\n")

In [178]:
import yaml

# Define the data as a Python dictionary
data = {
    "training.set": "train_sequential.txt",
    "test.set": "test_sequential.txt",
    # "training.set": "./dataset/amazon-beauty/train.txt",
    # "test.set": "./dataset/amazon-beauty/test.txt",
    "model": {
        "name": "CL4SRec",
        "type": "sequential"
    },
    "item.ranking.topN": [10, 20],
    "embedding.size": 64,
    "max.epoch": 500,
    "batch.size": 256,
    "learning.rate": 0.001,
    "reg.lambda": 0.0001,
    "max.len": 50,
    "CL4SRec": {
        "n_blocks":2,
        "drop_rate":0.2,
        "n_heads":1, 
        "aug_type": 0, 
        "aug_rate": 0.5,
        "cl_rate": 0.05
    },
    "output": "./results/"
}

# Save this dictionary as a YAML file
yaml_file_path = "config.yaml"
with open(yaml_file_path, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

# Load the YAML file to verify content
with open(yaml_file_path, "r") as file:
    conf = yaml.safe_load(file)

print("Loaded data from YAML file:", conf)

Loaded data from YAML file: {'CL4SRec': {'aug_rate': 0.5, 'aug_type': 0, 'cl_rate': 0.05, 'drop_rate': 0.2, 'n_blocks': 2, 'n_heads': 1}, 'batch.size': 256, 'embedding.size': 64, 'item.ranking.topN': [10, 20], 'learning.rate': 0.001, 'max.epoch': 500, 'max.len': 50, 'model': {'name': 'CL4SRec', 'type': 'sequential'}, 'output': './results/', 'reg.lambda': 0.0001, 'test.set': 'test_sequential.txt', 'training.set': 'train_sequential.txt'}


In [179]:
import torch
import torch.nn as nn
import numpy as np
from base.seq_recommender import SequentialRecommender
from util.sampler import next_batch_sequence
from model.sequential.SASRec import SASRec_Model
from util.loss_torch import l2_reg_loss,InfoNCE,batch_softmax_loss
from data.augmentor import SequenceAugmentor


# Paper: Contrastive Learning for Sequential Recommendation, ICDE'22

class CL4SRec(SequentialRecommender):
    def __init__(self, conf, training_set, test_set):
        super(CL4SRec, self).__init__(conf, training_set, test_set)
        args = self.config['CL4SRec']
        block_num = int(args['n_blocks'])
        drop_rate = float(args['drop_rate'])
        head_num = int(args['n_heads'])
        self.aug_type = int(args['aug_type'])
        self.aug_rate = float(args['aug_rate'])
        self.cl_rate = float(args['cl_rate'])
        self.model = SASRec_Model(self.data, self.emb_size, self.max_len, block_num,head_num,drop_rate)
        initializer = nn.init.xavier_uniform_
        self.model.item_emb = nn.Parameter(initializer(torch.empty(self.data.item_num + 2, self.emb_size)))
        self.rec_loss = torch.nn.BCEWithLogitsLoss()

    def train(self):
        model = self.model.cuda()
        optimizer = torch.optim.Adam(model.parameters(), lr=self.lRate)
        for epoch in range(self.maxEpoch):
            model.train()
            #self.fast_evaluation(epoch)
            for n, batch in enumerate(next_batch_sequence(self.data, self.batch_size,max_len=self.max_len)):
                seq, pos, y, neg_idx, seq_len = batch
                seq_emb = model.forward(seq, pos)
                if self.aug_type == 0: #crop
                    aug_seq1, aug_pos1, aug_len1 = SequenceAugmentor.item_crop(seq, seq_len,self.aug_rate)
                    aug_seq2, aug_pos2, aug_len2 = SequenceAugmentor.item_crop(seq, seq_len, self.aug_rate)
                    aug_emb1 = model.forward(aug_seq1, aug_pos1)
                    aug_emb2 = model.forward(aug_seq2, aug_pos2)
                    cl_emb1 = [aug_emb1[i, last - 1, :].view(-1, self.emb_size) for i, last in enumerate(aug_len1)]
                    cl_emb2 = [aug_emb2[i, last - 1, :].view(-1, self.emb_size) for i, last in enumerate(aug_len2)]
                elif self.aug_type == 1: #reorder
                    aug_seq1 = SequenceAugmentor.item_reorder(seq, seq_len, self.aug_rate)
                    aug_seq2 = SequenceAugmentor.item_reorder(seq, seq_len, self.aug_rate)
                    aug_emb1 = model.forward(aug_seq1, pos)
                    aug_emb2 = model.forward(aug_seq2, pos)
                    cl_emb1 = [aug_emb1[i,last-1,:].view(-1,self.emb_size) for i,last in enumerate(seq_len)]
                    cl_emb2 = [aug_emb2[i,last-1,:].view(-1,self.emb_size) for i,last in enumerate(seq_len)]
                else : # item mask
                    aug_seq1 = SequenceAugmentor.item_mask(seq, seq_len, self.aug_rate,self.data.item_num+1)
                    aug_seq2 = SequenceAugmentor.item_mask(seq, seq_len, self.aug_rate,self.data.item_num+1)
                    aug_emb1 = model.forward(aug_seq1, pos)
                    aug_emb2 = model.forward(aug_seq2, pos)
                    cl_emb1 = [aug_emb1[i, last - 1, :].view(-1, self.emb_size) for i, last in enumerate(seq_len)]
                    cl_emb2 = [aug_emb2[i, last - 1, :].view(-1, self.emb_size) for i, last in enumerate(seq_len)]
                cl_loss = self.cl_rate * InfoNCE(torch.cat(cl_emb1, 0), torch.cat(cl_emb2, 0), 1,True)
                rec_loss = self.calculate_loss(seq_emb, y, neg_idx, pos)
                batch_loss = rec_loss+ l2_reg_loss(self.reg, model.item_emb)+cl_loss
                # Backward and optimize
                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()
                if n % 50==0:
                    print('training:', epoch + 1, 'batch', n, 'batch_loss:', batch_loss.item(), 'rec_loss:', rec_loss.item())
            model.eval()
            self.fast_evaluation(epoch)

    def calculate_loss(self, seq_emb, y, neg,pos):
        y_emb = self.model.item_emb[y]
        neg_emb = self.model.item_emb[neg]
        pos_logits = (seq_emb * y_emb).sum(dim=-1)
        neg_logits = (seq_emb * neg_emb).sum(dim=-1)
        pos_labels, neg_labels = torch.ones(pos_logits.shape).cuda(), torch.zeros(neg_logits.shape).cuda()
        indices = np.where(pos != 0)
        loss = self.rec_loss(pos_logits[indices], pos_labels[indices])
        loss += self.rec_loss(neg_logits[indices], neg_labels[indices])
        return loss


    def predict(self,seq, pos,seq_len):
        with torch.no_grad():
            seq_emb = self.model.forward(seq,pos)
            last_item_embeddings = [seq_emb[i,last-1,:].view(-1,self.emb_size) for i,last in enumerate(seq_len)]
            score = torch.matmul(torch.cat(last_item_embeddings,0), self.model.item_emb.transpose(0, 1))
        return score.cpu().numpy()

In [180]:
from data.loader import FileIO


class SELFRec(object):
    def __init__(self, config):
        self.social_data = []
        self.feature_data = []
        self.config = config
        self.training_data = FileIO.load_data_set(config['training.set'], config['model']['type'])
        self.test_data = FileIO.load_data_set(config['test.set'], config['model']['type'])

        self.kwargs = {}
#         if config.contain('social.data'):
#             social_data = FileIO.load_social_data(self.config['social.data'])
#             self.kwargs['social.data'] = social_data
        # if config.contains('feature.data'):
        #     self.social_data = FileIO.loadFeature(config,self.config['feature.data'])
        print('Reading data and preprocessing...')

    def execute(self):
        # import the model module
#         import_str = f"from model.{self.config['model']['type']}.{self.config['model']['name']} import {self.config['model']['name']}"
#         exec(import_str)
        recommender = f"{self.config['model']['name']}(self.config,self.training_data,self.test_data,**self.kwargs)"
        return eval(recommender)

In [181]:
model = 'CL4SRec'
conf = ModelConf('config.yaml')
CL4SRec = SELFRec(conf).execute()

Reading data and preprocessing...


In [182]:
CL4SRec.execute()

Model: CL4SRec
Training Set: /workspace/viettel-ai-challenge-track-ds/SELFRec/train_sequential.txt
Test Set: /workspace/viettel-ai-challenge-track-ds/SELFRec/test_sequential.txt
Embedding Dimension: 64
Maximum Epoch: 500
Learning Rate: 0.001
Batch Size: 256
Regularization Parameter: 0.0001
Specific parameters: aug_rate:0.5  aug_type:0  cl_rate:0.05  drop_rate:0.2  n_blocks:2  n_heads:1
Training Set Size: (sequence number: 42670, item number: 78343)
Initializing and building model...
Training Model...
training: 1 batch 0 batch_loss: 1.639357328414917 rec_loss: 1.3865199089050293
training: 1 batch 50 batch_loss: 1.6091389656066895 rec_loss: 1.3603925704956055
training: 1 batch 100 batch_loss: 1.5093053579330444 rec_loss: 1.2514028549194336
training: 1 batch 150 batch_loss: 1.461295247077942 rec_loss: 1.204667091369629
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
--------------------------------------------------------------------------------


KeyboardInterrupt: 

In [35]:
CL4SRec.print_model_info()

Model: CL4SRec
Training Set: /workspace/viettel-ai-challenge-track-ds/SELFRec/train_sequential.txt
Test Set: /workspace/viettel-ai-challenge-track-ds/SELFRec/test_sequential.txt
Embedding Dimension: 64
Maximum Epoch: 500
Learning Rate: 0.001
Batch Size: 256
Regularization Parameter: 0.0001
Specific parameters: aug_rate:0.5  aug_type:0  cl_rate:0.05  drop_rate:0.2  n_blocks:2  n_heads:1
Training Set Size: (sequence number: 38858, item number: 67532)


In [36]:
predict_df = pd.read_csv('../data/public_testset.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
predict_df.head()

,user_id,item_id_1,item_id_2,item_id_3,item_id_4,item_id_5,item_id_6,item_id_7,item_id_8,item_id_9,...,item_id_991,item_id_992,item_id_993,item_id_994,item_id_995,item_id_996,item_id_997,item_id_998,item_id_999,item_id_1000
0,NcVPv81tzp,NYRp1o1YKD,nyDJiBSznn,tMMpf5S62m,pYCyajy1Am,IQYqanXAvK,K0nTlcclVU,aTcR4pfQrM,md0Mi2Y3PN,BGpHuuj0eD,...,Xi7bQ3D5Nd,DnHOgmVXrS,6Uns6g05f7,20kEvp8BVk,vVjXZVhRDe,Z2h9Oni93D,uHuaWLAHdv,RYpgK2XiNS,cFfT0AEdOb,WCm27PLRAp
1,C5ihgxLcrb,1bWoiCKvUG,uBZeAeJAp1,wz3fiRi1XX,kcLQeDE9Y9,XoyZaOJ2hQ,cfkOAeEV6p,lywvWPArBx,41uJKXzbXd,1kVkVTBnhQ,...,j387AnkQH6,HkvDRz6oXT,fSv7hmSvI4,LHh3iij8he,mf2eKsaafi,rKwmw637oi,nXcPqalddS,CiGuUIbCEO,5QBxhJ93BT,bo1CjxPHpP
2,ljMBWT9UXW,1M0GtQfkUA,iytV2FTVjK,G2Sufv4xlY,AS24sgQN69,U1uDxeAwr4,yniLMM6GB7,ziFGlTdzPp,BddfNX8HRd,S9tgPjbv1a,...,VDBauWNjJF,NSMr3KjVnR,aJUzJBXEfY,tqC65yziRM,4zDIIH126P,M1tDsFuWBH,PTvaMJlQyK,HfvagfNvGp,XUWoQVdsmN,EdwIvwMRoZ
3,WZXCQtOWJt,2J7erZG9Af,S0Q4XoexaS,hCZTh4mNx8,dUXbCsX8et,mA7s8zgFMR,YFdqrb8OGd,C9FNEwKKd8,mtsRoBIDbs,1b49EFCcJG,...,dwVDUM6ngC,IFwPdofoEP,sFGjZgyZnH,F10yUBudDF,XK550LREfU,3uZFqvw0zz,Dto8H87MIx,tiQGe5Ye7v,CzHVYocfQY,wc7rJoeZRQ
4,xlB2rvpQKu,fZIHEceEkK,FurfEc4uKO,4FoN0bcH4x,cslhbAk2FI,Wi8OmeoDhH,uHdZ7W1MOy,xqyiojw3KO,Hj1UDRyfAc,vSeyk7gadz,...,2IpMrt6ieN,odYnPAZSJv,5FV9Qd8ABf,se19TPTKGt,tXTRyaQqsA,8KqODVTepQ,1iqudkzZxj,H7ZyQBRUQq,0NDBaPifhZ,u0altqehZS


In [93]:
from base.recommender import Recommender
from data.sequence import Sequence
from util.algorithm import find_k_largest
from util.evaluation import ranking_evaluation
from util.sampler import next_batch_sequence_for_test

def test(self):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        print(f'\rProgress: [{"+" * ratenum}{" " * (50 - ratenum)}]{ratenum * 2}%', end='', flush=True)

    rec_list = {}
    for n, batch in enumerate(next_batch_sequence_for_test(self.data, self.batch_size, max_len=self.max_len)):
        seq, pos, seq_len = batch
        seq_start = n * self.batch_size
        seq_end = (n + 1) * self.batch_size
        seq_names = [seq_full[0] for seq_full in self.data.original_seq[seq_start:seq_end]]
        candidates = self.predict(seq, pos, seq_len)
        for name, res in zip(seq_names, candidates):
            ids, scores = find_k_largest(1000, res)
            
            item_names = [self.data.id2item[iid] for iid in ids if iid != 0 and iid <= self.data.item_num]
            rec_list[name] = list(zip(item_names, scores))
        if n % 100 == 0:
            process_bar(n, self.data.raw_seq_num / self.batch_size)
    process_bar(self.data.raw_seq_num, self.data.raw_seq_num)
    print('')
    return rec_list
rec_list = test(CL4SRec)

Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%


In [125]:
user_map_test = {UserId: index for index, UserId in enumerate(user_list_denoise)}
user_map_test = pd.DataFrame(list(user_map_test.items()), columns=['UserId', 'index']).merge(
    session.groupby("TrueUser").agg({"UserId":list, "ItemId": list}), 
    left_on="UserId",
    right_on="TrueUser")
user_map_test = user_map_test.drop(columns=["index", "ItemId"]).rename(columns={"UserId_x": "UserId", "UserId_y": "index"})
user_map_test

,UserId,index
0,AgVHPOXsl9,[44191]
1,XtXFj3GXVv,[30046]
2,v0tAgIudEG,[9856]
3,UUFL2eB35z,[15002]
4,pf2reuvkUz,"[3165, 3166, 3167, 3168, 3169]"
...,...,...
36015,LyC2I87Q7T,[27954]
36016,lccT8avPqr,"[29521, 29522, 29523]"
36017,IDtxsQrxIV,[23261]
36018,srB0ypqWRm,"[666, 667, 668, 669, 670, 671, 672, 673, 674, ..."


In [134]:
import numpy as np
from tqdm import tqdm
# Precompute mappings for faster lookups
user_map_dict = user_map_test.set_index('UserId')['index'].to_dict()
item_map_dict = item_map.set_index('ItemId')['index'].astype(str).to_dict()

data = []  # Result storage

cnt = 0
for user_id in tqdm(test_user_id):
    # Look up user index using the precomputed dictionary
    user_index = user_map_dict.get(user_id)
    if not user_index:
        continue  # Skip if user index is not found
    user_index.reverse()
    # Retrieve item names and candidate pairs for this user
    item_names = predict_df[predict_df.user_id == user_id].values[0][1:]
    candidates = None
    for index in user_index:
        index = str(index)
        try:
            candidates = rec_list[index]
            break
        except:
            continue
    if candidates is None:
        cnt += 1
        data.append([user_id])
        continue
    
    candidate_index = {str(candidate[0]): candidate[1] for candidate in candidates}  # Map item index to score

    # Create score list for item_names by checking candidate_index
    scores = [
        candidate_index.get(item_map_dict.get(item), 1e-8) if item in item_map_dict else 1e-8
        for item in item_names
    ]
    
    # Sort items based on score in descending order
    sorted_list = sorted(zip(item_names, scores), key=lambda x: x[1], reverse=True)
    
    # Append user_id and sorted items to data
    data.append([user_id] + [i[0] for i in sorted_list])

#     print(data)
#     break

100%|██████████| 2817/2817 [00:02<00:00, 950.31it/s]


In [135]:
cnt

151

In [136]:
pd.DataFrame(data).fillna("0").to_csv('predict.csv', index = False, header=False)

In [137]:
!zip predict.zip predict.csv

updating: predict.csv (deflated 27%)


In [108]:
pd.DataFrame(data).fillna("0")

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,NcVPv81tzp,mwOc6vrVPi,oNn8dJXhM8,3tljNC7rIT,G5ph8DYOJT,y9Xw55rEhr,o9yXO6Ak9I,Y65rfz4uVb,VbS4iRieJX,oxsy6DmtHi,...,av2irHMO5f,q4bgY20S3L,xRrrBsbzL7,IQYqanXAvK,tIKJSzyuSt,ATrOh0re7K,gzvOLHhGdz,mCFiC4becE,Xd4kWZ81Ud,bcgeiUPdnd
1,C5ihgxLcrb,PfQbz0wOPX,u0KbwAhFgw,muNNVgPp6B,zAm419EocZ,kk3smPWhZf,7NudKUsc5y,Bm335Ij3FP,4No0gAp40y,1bWoiCKvUG,...,LHh3iij8he,mf2eKsaafi,rKwmw637oi,nXcPqalddS,CiGuUIbCEO,5QBxhJ93BT,bo1CjxPHpP,OuXKg3dEV6,k5xbDawW4n,qO0GeKulOn
2,ljMBWT9UXW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,WZXCQtOWJt,zBaQw1EQgW,2J7erZG9Af,S0Q4XoexaS,hCZTh4mNx8,dUXbCsX8et,mA7s8zgFMR,YFdqrb8OGd,C9FNEwKKd8,mtsRoBIDbs,...,iZ8tT7ZsaN,WeES3Ooj3f,In5DecoXSo,UxV8tVey2n,cBVWr65FV2,xGi5iqsvqC,KHl3ESgRef,leIUAqtxH5,1VHFgCcKzG,PBKJAIHrk1
4,xlB2rvpQKu,fZIHEceEkK,FurfEc4uKO,4FoN0bcH4x,cslhbAk2FI,Wi8OmeoDhH,uHdZ7W1MOy,xqyiojw3KO,Hj1UDRyfAc,vSeyk7gadz,...,i2IQYW9wu1,9eaM2IdOik,9nnxVOSStv,DJ1UNbmPJr,sV5ZP8nBKe,Xr5LerfZ0P,TNCS6O3iSq,2Mqq3N13HR,BAVLYDttpI,rNGK9hb1gW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812,51ANpugrpL,S4xaXwp7iP,XovMQEyq44,JHFpmP1jTQ,iME0Muljhc,ggY7xPOpcd,wYyxCM62Vf,rwdF7Z8Lqd,xheWrneGCx,TNwRhZvnDu,...,88n4BJ0Jcz,GlUKC50HcM,FNoFVV3iPt,bBMj9AGqqS,vILgGw98pn,bGMCLeABTX,iRyT9YX94j,5Ud5NkBg0X,Lc9bJ81t1K,3Bt4TPSa4S
2813,PpBVsb2nMx,EYbGS5M7z4,pAqFb3i2ZL,1RI5B7JsfJ,xUDRO5klBj,6gbEkevMZQ,VldjdED29h,rzE3O0nubX,p3XLGxRnhF,vzf2kEW2IZ,...,4LXkg6FURV,Z6iT2Pnw3i,HJa8Jp8Uwj,IvdCqDs2Dq,JA1LqZSMx0,HLFzBYKhhQ,bcl22AXA6X,FIN0HplPu4,VFaGpZcVBQ,59zPRxT8n8
2814,EkulSxnt42,R91En1VJv4,N5nKpygoaP,MAmamNFSU9,SrB6LifA8L,A2v66utrO9,VBCWSphOfG,K3TUntaKwi,F6yFWVSmCx,6jy1DwPo7L,...,6lds5fb2s2,QycEIlERUq,jNUISWueIr,7L49Mdg1OF,1TcYMKFer9,Lr3tZQ6ErB,p1UtVZevlM,m5YDyKjlZv,tZUW1y3xY7,yaiEUJz8c1
2815,eflbPkaBp9,NECs6b672k,QgcWlvxmht,w3ilCPMvbl,ltnrq6mncd,fgONK18lzp,Mkdazz84pL,SmsSbsqaZm,5n9wFTyqx9,CAlYbRQhla,...,CthVhk4IIv,7qvhONF93A,akb1Xfk51Y,jb8LikQQZF,YONnqjo9Go,TqtytuKGFc,zagEZFzciz,dCvHtITENn,fvvu9onv0u,8oTrOiEXbQ
